In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import model_evaluation_functions

In [2]:
%load_ext autoreload
%autoreload 2

# Load and clean prediciton data

In [22]:
df_ground_truth = pd.read_csv('../annotations/ground_truth_boat_frames_coco.csv', index_col=0, sep=';')
df_yolo_predictions = pd.read_csv('../annotations/yolov8x_full_data_train_video_labels.csv', index_col=0, sep=';')

df_ground_truth['datetime'] = pd.to_datetime(df_ground_truth.datetime)
df_ground_truth['date'] = pd.to_datetime(df_ground_truth.datetime.dt.date, format='%Y-%m-%d')
df_yolo_predictions['datetime'] = pd.to_datetime(df_yolo_predictions['datetime'])
df_yolo_predictions['date'] = pd.to_datetime(df_yolo_predictions.datetime.dt.date, format='%Y-%m-%d')

df_ground_truth = df_ground_truth[df_ground_truth.date.isin(df_yolo_predictions.date.unique())]

print('shape of loaded data', df_ground_truth.shape, df_yolo_predictions.shape)
print('Ground truth data:')
print(df_ground_truth.groupby(['date', 'camera_id']).datetime.count())
print('YOLO predictions data:')
print(df_yolo_predictions.groupby(['date', 'camera_id']).datetime.count())

# crop bounding boxes from right side of camera 2 field of view
# df_yolo_predictions.drop(index=df_yolo_predictions[(df_yolo_predictions.camera_id == 2) & (df_yolo_predictions.x > 1800)].index, inplace=True)
# df_ground_truth.drop(index=df_ground_truth[(df_ground_truth.camera_id == 2) & (df_ground_truth.x > 1800)].index, inplace=True)

df_ground_truth.set_index('filename', inplace=True)
df_yolo_predictions.set_index('filename', inplace=True)

df_ground_truth.shape, df_yolo_predictions.shape

shape of loaded data (11492, 11) (116558, 11)
Ground truth data:
date        camera_id
2023-07-07  2            6500
2023-07-08  1            4992
Name: datetime, dtype: int64
YOLO predictions data:
date        camera_id
2023-07-07  2            109305
2023-07-08  1              7253
Name: datetime, dtype: int64


((11492, 10), (116558, 10))

In [23]:
# transform coords to xmin, ymin, xmax, ymax
df_ground_truth['xmin'] = (df_ground_truth['x'] - df_ground_truth['w']/2).astype(int)
df_ground_truth['ymin'] = (df_ground_truth['y'] - df_ground_truth['h']/2).astype(int)
df_ground_truth['xmax'] = (df_ground_truth['x'] + df_ground_truth['w']/2).astype(int)
df_ground_truth['ymax'] = (df_ground_truth['y'] + df_ground_truth['h']/2).astype(int)
df_yolo_predictions['xmin'] = (df_yolo_predictions['x'] - df_yolo_predictions['w']/2).astype(int)
df_yolo_predictions['ymin'] = (df_yolo_predictions['y'] - df_yolo_predictions['h']/2).astype(int)
df_yolo_predictions['xmax'] = (df_yolo_predictions['x'] + df_yolo_predictions['w']/2).astype(int)
df_yolo_predictions['ymax'] = (df_yolo_predictions['y'] + df_yolo_predictions['h']/2).astype(int)

In [24]:
# filter out predicitons in the bank on camera 1; use a line to define the bank in the camera 1 field of view
point1_cam01_bank = (489, 591)
point2_cam01_bank = (1499, 875)
slope_cam01_bank = (point2_cam01_bank[1] - point1_cam01_bank[1]) / (point2_cam01_bank[0] - point1_cam01_bank[0])
intercept_cam01_bank = point1_cam01_bank[1] - slope_cam01_bank * point1_cam01_bank[0]

def under_the_bank_apply(row):
    if row.camera_id == 1 and row.x >= point1_cam01_bank[0] and row.x <= point2_cam01_bank[0]:
        if row.y > (slope_cam01_bank * row.x + intercept_cam01_bank):
            return True        
    return False
        
df_yolo_predictions['under_the_bank'] = df_yolo_predictions.apply(under_the_bank_apply, axis=1)
print('How many prediction were under the bank?')
print(df_yolo_predictions['under_the_bank'].value_counts())
df_yolo_predictions = df_yolo_predictions[df_yolo_predictions.under_the_bank == False].copy()
print('YOLO predictions filtered data:')
print(df_yolo_predictions.groupby(['date', 'camera_id']).datetime.count())

How many prediction were under the bank?
under_the_bank
False    116552
True          6
Name: count, dtype: int64
YOLO predictions filtered data:
date        camera_id
2023-07-07  2            109305
2023-07-08  1              7247
Name: datetime, dtype: int64


# Evaluation of precision and recall for detected frame_ids

In [21]:
# before cam2 >1800 filtration
{'f1': 0.18191988154972444,
 'recall': 0.9799716412619638,
 'precision': 0.10026657962026005,
 'iou': 0.7545292423621479}

{'mAP_0.5_11points': 0.25194466,
 'mAP_0.5_all_points': 0.2467102,
 'mAP_0.5_0.95_101points': 0.13477196}

# after cam2 >1800 filtration
{'f1': 0.24955827838277922,
 'recall': 0.97844788553835,
 'precision': 0.143017868960953,
 'iou': 0.7568439903162304}

{'mAP_0.5_11points': 0.33203986,
 'mAP_0.5_all_points': 0.3046532,
 'mAP_0.5_0.95_101points': 0.16641793}

{'f1': 0.24955827838277922,
 'recall': 0.97844788553835,
 'precision': 0.143017868960953,
 'iou': 0.7568439903162304}

In [30]:
df_evaluation, total_evaluation_dict = model_evaluation_functions.evaluate_model_frame_ids_score(df_ground_truth, df_yolo_predictions)
total_evaluation_dict

  0%|          | 0/135 [00:00<?, ?it/s]

100%|██████████| 135/135 [00:17<00:00,  7.80it/s]


{'f1': 0.18191988154972444,
 'recall': 0.9799716412619638,
 'precision': 0.10026657962026005,
 'iou': 0.7545292423621479}

In [31]:
df_evaluation.to_csv('data_evaluation_full_data_with_false_positive.csv', sep=';')

In [26]:
model_evaluation_functions.evaluate_mAP(df_ground_truth, df_yolo_predictions)

  0%|          | 0/84 [00:00<?, ?it/s]

100%|██████████| 84/84 [01:09<00:00,  1.21it/s]


{'mAP_0.5_11points': 0.25194466,
 'mAP_0.5_all_points': 0.2467102,
 'mAP_0.5_0.95_101points': 0.13477196}

In [33]:
df_ground_truth_cam1 = df_ground_truth[df_ground_truth['camera_id'] == 1]
df_prediction_cam_1 = df_yolo_predictions[df_yolo_predictions['camera_id'] == 1]

In [34]:
df_evaluation, total_evaluation_dict = model_evaluation_functions.evaluate_model_frame_ids_score(df_ground_truth_cam1, df_prediction_cam_1)
total_evaluation_dict

100%|██████████| 39/39 [00:06<00:00,  6.41it/s]


{'f1': 0.8003059143439837,
 'recall': 0.9681332236842105,
 'precision': 0.6820683661645423,
 'iou': 0.7782717660738925}

In [35]:
model_evaluation_functions.evaluate_mAP(df_ground_truth_cam1, df_prediction_cam_1)

100%|██████████| 33/33 [00:03<00:00,  9.08it/s]


{'mAP_0.5_11points': 0.66977364,
 'mAP_0.5_all_points': 0.6638641,
 'mAP_0.5_0.95_101points': 0.39705622}

## Use validation part of data and only a filter images with known ground truth

In [27]:
df_ground_truth_filtered = df_ground_truth[(df_ground_truth.date == '2023-07-07') | (df_ground_truth.date == '2023-07-08')].reset_index().set_index(['filename', 'frame_id'])
df_yolo_predictions_filtered = df_yolo_predictions.reset_index().set_index(['filename', 'frame_id'])
df_yolo_predictions_filtered = df_yolo_predictions_filtered.loc[df_ground_truth_filtered.index.intersection(df_yolo_predictions_filtered.index)]
df_ground_truth_filtered.shape, df_yolo_predictions_filtered.shape

((11492, 13), (12246, 14))

In [28]:
df_ground_truth_filtered = df_ground_truth_filtered.reset_index().set_index('filename')
df_yolo_predictions_filtered = df_yolo_predictions_filtered.reset_index().set_index('filename')
df_evaluation, total_evaluation_dict = model_evaluation_functions.evaluate_model_frame_ids_score(df_ground_truth_filtered, df_yolo_predictions_filtered)
total_evaluation_dict

  4%|▎         | 3/84 [00:00<00:18,  4.42it/s]

100%|██████████| 84/84 [00:14<00:00,  5.68it/s]


{'f1': 0.9898845224241339,
 'recall': 0.9799716412619638,
 'precision': 1.0,
 'iou': 0.7545292423621482}

In [29]:
model_evaluation_functions.evaluate_mAP(df_ground_truth_filtered, df_yolo_predictions_filtered)

  1%|          | 1/84 [00:00<00:11,  7.47it/s]

100%|██████████| 84/84 [00:18<00:00,  4.42it/s]


{'mAP_0.5_11points': 0.881029,
 'mAP_0.5_all_points': 0.89613456,
 'mAP_0.5_0.95_101points': 0.4728706}

# Confidence values evaluation

In [ ]:
confidence_threshold_evaluation_dict = dict()
for confidence_threshold in [0.25, 0.5, 0.75, 0.9]:
    print('Confidence threshold', confidence_threshold)
    df_evaluation, total_evaluation_dict = model_evaluation_functions.evaluate_model_frame_ids_score(df_ground_truth, df_yolo_predictions[df_yolo_predictions.confidence >= confidence_threshold])    
    print(total_evaluation_dict)
    # df_evaluation.to_csv(f'data_evaluation_conf{confidence_threshold}.csv', sep=';')
    confidence_threshold_evaluation_dict[confidence_threshold] = total_evaluation_dict

In [ ]:
plt.plot(confidence_threshold_evaluation_dict.keys(), [total_evaluation_dict['recall'] for total_evaluation_dict in confidence_threshold_evaluation_dict.values()], 'x-')
plt.xlabel('Confidence threshold')
plt.ylabel('Recall')

In [ ]:
plt.plot(confidence_threshold_evaluation_dict.keys(), [total_evaluation_dict['f1'] for total_evaluation_dict in confidence_threshold_evaluation_dict.values()], 'x-')
plt.xlabel('Confidence threshold')
plt.ylabel('F1')

In [ ]:
plt.plot([total_evaluation_dict['recall'] for total_evaluation_dict in confidence_threshold_evaluation_dict.values()], [total_evaluation_dict['precision'] for total_evaluation_dict in confidence_threshold_evaluation_dict.values()], 'x-')
plt.xlabel('Recall')
plt.ylabel('Precision')

In [ ]:
plt.plot(confidence_threshold_evaluation_dict.keys(), [total_evaluation_dict['iou'] for total_evaluation_dict in confidence_threshold_evaluation_dict.values()], 'x-')
plt.xlabel('Confidence threshold')
plt.ylabel('IOU')